# Tutorial BuildME

This tutorial shows the most important functionalities of the BuildME framework using a mix of interactive code cells and text cells. The tutorial walks the user through the most important steps shown also in the `main.py` file. The four most important stages shown in this tutorial include:
1. Setting up the framework
1. Simulating energy demand
1. Simulating material demand
1. Processing the results

## 1. Setting up the framework

### Imports

First, we import the BuildME framework (and the `os` package which allows us to change the current working directory).

In [ ]:
import os

if os.path.basename(os.getcwd()) != 'BuildME':
    os.chdir('../..')

from BuildME import pre, simulate

### Selection of archetypes

As the next step, we need to define the type of building we want to simulate. We can do that by creating a variable `debug_combinations`. This variable should contain information such as region, occupation and other aspects. The possible combinations of the different aspects are listed in the `combinations` variable in `settings.py`. 

Let's say we want to simulate building located in the **US**. The building is a single family house (**SFH**), typical in terms of energy standards (**standard**) and with no special assumptions on resource efficiency (**RES0**). The energy requirements for this building are influenced by climate, so we specify the time (year 2015) and location of the building (Miami, climate region **1A**). The cooling demand of this building is fulfilled solely using the **HVAC** system (an alternative is to use the mixed-mode ventilation system - MMV).

In [ ]:
debug_combinations = {
    'USA':
        {'occupation': ['SFH'],
         'energy standard': ['standard'],
         'RES': ['RES0'],
         'climate_region': ['1A'],
         'climate_scenario': ['2015'],
         'cooling': ['HVAC']}}

### Checking EnergyPlus compatibility

The next step is to check the EnergyPlus version - the one installed by the used locally should match the one that is used by BuildME. 

In [ ]:
pre.validate_ep_version()

### Creating MMV variants (or at least an empty xlsx file)

If the user has selected 'MMV' as the cooling type, then the next step is to create a MMV variant out of the archetype that was chosen for simulation. Even in case the user has selected 'HVAC' as the cooling type, the MMV precalculation routine needs to run to create an empty `replace_mmv.xlsx` file in the `data` folder. Creating this file ensures that the simulation can proceed in further steps of the workflow.

In [ ]:
pre.create_mmv_variants(comb=debug_combinations)

### Preparing the files for simulations


Based on the previously defined archetype (variable `debug_combinations`), we create the files to be used in energy and/or material simulations. The first step is to create an array `simulation_files` with information about each building instance to be simulated. Each element of this array includes a list with necessary information about a building, such as the location of the .EPW climate file or the IDF building file. 

In addition, we create a variable `run` which is a unique identifier of this model run (created based on the timestamp).

In [ ]:
simulation_files, run = simulate.create_combinations(debug_combinations)

The next step ensures that folders created in previous model runs do not interfere with the current simulation workflow. In case such folders are identified, the `nuke_folders()` function deletes them.

In [ ]:
simulate.nuke_folders(simulation_files, run)  # deletes only the folder with the case you try to simulate

Finally, based on the information stored in the `simulation_files` variable, the `copy_scenario_files()` function creates new folders in the `tmp` folder. 

The folders are then populated with weather files (.EPW) and building files (.IDF) chosen depending on the archetypes and climates selected in the 'debug_combinations' step. The weather files need to be created by the user using dedicated software, e.g. Meteonorm. If the files are not found, BuildME uses a dummy weather file for Miami, Florida, [made available by the U.S. Deparment of Energy](https://www.energycodes.gov/prototype-building-models).

This step includes an essential part of the BuildME framework: substituting the so-called *replacement* strings. Inside the `copy_scenario_files()` function, we find `apply_obj_name_change()` routine applied to energy standard and resource efficiency scenarios. The routine substitutes the '-en-std-replaceme' and '-res-replaceme' strings included in the names of some materials by strings specific to the simulated building (e.g. 'standard' and 'RES0' like in our example). This way, BuildME automatizes the implementation of various energy standards which use e.g. insulation layers of different thickness.

In [ ]:
simulate.copy_scenario_files(simulation_files, run)

After performing the step described above, the respective folders contain the EPW and IDF files necessary for the simulation. The IDF file called 'in.idf' is ready for simulation, e.g. all the 'replaceme' strings have been substituted. This means that the file could also be used to manually simulate energy demand using EnergyPlus GUI called EP-Launch.exe or similar. Please note that if the `copy_scenario_files()` function was to be omitted, then the 'replaceme' strings would *not* be substituted and the simulation would fail because these names would not match any constructions defined in the IDF file.

## 2. Simulating energy demand

At this point, we can perform the energy simulation using EnergyPlus. Please note that this step takes more time to execute,  depending on the chosen archetype and cooling strategy; it can take between around 20 seconds (e.g., for SFH archetype with HVAC cooling) and more than 15 minutes (e.g., MFH archetype with MMV cooling). After the simulation is done, a prinout message appears below, indicating the number of seconds it took to simulate one building (= one iteration) e.g., 22.26s/it.

In [ ]:
simulate.calculate_energy()

After performing the energy simulation, the respective folders are filled with simulation files (typically more than 30). The most important of these are:
- `eplusout.err` which informs us about critical errors (in case the simulation did not succeed) or warnings;
- `eplusout.csv` which includes variable values, as listed in the *Output:Variable* object of the IDF file;
- `eplusmtr.csv` which includes metered values, as listed in the *Output:Meter* object of the IDF file.

## 3. Simulating material demand

To calculate the amount of material used in the building, BuildME checks all the constructions used in the building, lists all the layers of materials and adds up their volumes, which are then converted to mass. The materials are then grouped into categories such as 'concrete', 'brick', 'paper and cardboard' etc., as indicated by the `odym_materials` variable found in the `settings.py` file.

After performing this step, a file `materials.csv` appears in the respective folder. The file contains the material categories and their mass (kg). On the bottom, the floor area (m<sup>2</sup>) is also given, which can be used to calculate material intensity.

In [ ]:
simulate.calculate_materials()

## 4. Processing the results

After all the simulations have been performed, the results can be processed and structured. 

The first post-processing step includes the creation of an `energy_demand.csv` file, which lists the various energy use categories (heating, cooling, lighting, equipment) and their energy use (J). The energy values are also saved to a variable `res_energy` which is used in the following steps. Similarly, we need a variable containing the information about material demand, which is why `res_mat` is also created.

In [ ]:
res_energy = simulate.collect_energy(simulation_files)
res_mat = simulate.load_material(simulation_files)

TO DO: Explain what happens in the two steps below:
- DHW is added manually.
- What are the units? (seems like MJ, but maybe it would be more natural to have kWh)
- What is the importance of scenarios? It is mentioned in functions such as `divide_by_area`, `disaggregate_scenario_str`, and `weighing_climate_region`

In [ ]:
simulate.save_ei_result(run, res_energy, res_mat)

In [ ]:
simulate.save_mi_result(run, res_mat)